<a href="https://colab.research.google.com/github/AlexandruBrezuleanu/Advanced-Topics-in-Neural-Networks-Template-2023/blob/main/Tema3_RN_Brezuleanu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import Tensor
import time
import random
import pandas as pd
import sys
import torch.nn.functional as TF

#m=20

#X = torch.rand((m,784))
W1 = torch.rand((784,100))/10
W2 = torch.rand((100,10))/10
#y = torch.zeros((m,10))

b1 = torch.rand((100))/10
b2 = torch.rand((10))/10

lr = 0.01



In [ ]:
def sigmoid_function(x):
  return 1/(1+torch.exp(-x))

def forward(X:Tensor, W1:Tensor, W2:Tensor, b1:Tensor, b2:Tensor):
  z1 = X @ W1 + b1
  y_hat1 = sigmoid_function(z1)

  z2 = y_hat1 @ W2 + b2
  y_hat2 = sigmoid_function(z2)
  return y_hat2

def train_batch(X:Tensor, W1:Tensor, W2:Tensor, b1:Tensor, b2:Tensor, y_true: Tensor, lr:float):
  z1 = X @ W1 + b1
  y_hat1 = sigmoid_function(z1)
  z2 = y_hat1 @ W2 + b2
  y_hat2 = sigmoid_function(z2)
  error2 = y_hat2 - y_true
  error1 = y_hat1 * (1 - y_hat1) * (error2 @ W2.T)

  for index, line in enumerate(y_hat1):
    new_x = line.unsqueeze(0)
    new_y = error2[index].unsqueeze(0)
    diff_W2 = new_x.T * new_y
    W2 = W2 - lr * diff_W2
  for index, line in enumerate(X):
    new_x = line.unsqueeze(0)
    new_y = error1[index].unsqueeze(0)
    diff_W1 = new_x.T * new_y
    W1 = W1 - lr * diff_W1

  b2 = b2 - lr * error2.mean(axis=0)
  b1 = b1 - lr * error1.mean(axis=0)
  return W1,W2,b1,b2

def train_epoch(X:Tensor, W1:Tensor, W2:Tensor, b1:Tensor, b2:Tensor, y_true: Tensor, lr:float, batch_size):
  for i in range(0, X.shape[0], batch_size):
      x = X[i: i + batch_size]
      y = y_true[i: i + batch_size]
      W1,W2,b1,b2 = train_batch(x,W1,W2,b1,b2,y,lr)

  return W1,W2,b1,b2

def train_mlp(X:Tensor, W1:Tensor, W2:Tensor, b1:Tensor, b2:Tensor, y_true: Tensor, lr:float):
    epoch_size = 30
    batch_size = 100
    for epoch in range(epoch_size):
      W1,W2,b1,b2 = train_epoch(X,W1,W2,b1,b2,y_true,lr,batch_size)
    return W1,W2,b1,b2

In [ ]:
train_data = pd.read_csv('mnist_train.csv')
label = train_data['label']
label_train_tensor = torch.LongTensor(label)
train_tensor = torch.tensor(train_data.drop( 'label', axis = 1).values, dtype=torch.float)
train_tensor = train_tensor / (255 * 10)

y_train = torch.zeros((len(label), 10))
for index, val in enumerate(label):
  y_train[index][val] = 1


In [ ]:
test_data = pd.read_csv('mnist_test.csv')
label_test = test_data['label']
label_test_tensor = torch.LongTensor(label_test)
test_tensor = torch.tensor(test_data.drop( 'label', axis = 1).values, dtype=torch.float)
test_tensor = test_tensor / (255 *10)

y_test = torch.zeros((len(label_test), 10))
for index, val in enumerate(label_test):
  y_test[index][val] = 1

In [ ]:
%%time
W1,W2,b1,b2 = train_mlp(train_tensor, W1,W2,b1,b2,y_train,lr)

CPU times: user 4min 39s, sys: 419 ms, total: 4min 40s
Wall time: 4min 41s


In [ ]:
predicted_y = forward(train_tensor, W1,W2,b1,b2)

correct=0
for index, row in enumerate(y_train):
  if torch.argmax(row) == torch.argmax(predicted_y[index]):
    correct+=1

print("Accuracy on training set: " + str((100 * correct)/len(train_tensor)))

loss = TF.cross_entropy(predicted_y, label_train_tensor)

# Print the computed loss
print(f"Cross entropy loss on training set: {loss.item()}")

Accuracy on training set: 95.15
Cross entropy loss on training set: 1.5578364133834839


In [ ]:
predicted_y = forward(test_tensor, W1,W2,b1,b2)

correct=0
for index, row in enumerate(y_test):
  if torch.argmax(row) == torch.argmax(predicted_y[index]):
    correct+=1

print("Accuracy on validation set: " + str((100 * correct)/len(test_tensor)))

loss = TF.cross_entropy(predicted_y, label_test_tensor)

# Print the computed loss
print(f"Cross entropy loss on validation test: {loss.item()}")

Accuracy on validation set: 94.93
Cross entropy loss on validation test: 1.557747721672058
